# DB Fixes

Back up your database first!

After inserting entries in database, we needed some minor fixes to clean up data.
- remove slashes from identifier entries
- find where there are duplicate entries for a given arXiv identifier and remove first entries
- check which images have no matching metadata
- automatically download arXiv OAI data and add it into the database
- search arXiv metadata JSON file

There is also code for replacing a table from another database.

NB: could be bugs in this and might require manual setting up depending on dataset and metadata

## Setup

In [ ]:
import os
import xml.etree.ElementTree as ET
import sqlite3
import requests
import time
import json

Connect to database

In [ ]:
db_path = os.path.expanduser("~/data/db/arxiv_db_images.sqlite3")
db = sqlite3.connect(db_path)
c = db.cursor()

Check that we can access the table pragma info

In [ ]:
c.execute('PRAGMA TABLE_INFO({})'.format("metadata"))
info = c.fetchall()

print("\nColumn Info:\nID, Name, Type, NotNull, DefaultVal, PrimaryKey")
for col in info:
    print(col)

## Remove slashes

We also searched the SQLite database for any entries with special characters such as '/' that caused errors in the file insertion. In each arXiv category pre-2007, we removed the forward slash as this could potentially cause problems in how this data is read (this can be added back in as required).

This is done via SQLite command to replace the slash character with an empty string ('').

#### !!! MAKE SURE TO BACK UP DATABASE FIRST !!!

In [ ]:
sql = ("UPDATE metadata SET identifier=REPLACE(identifier, '/', '') WHERE identifier LIKE '%/%'")
c.execute(sql)
db.commit()

## Duplicate entries

There are a number of entries in the metadata table that are duplicates. This seems to be due to the way that arXiv maintains OAI metadata when there is a version update to a given article, or it could point to the way that `metha` downloads the data

In [ ]:
# look for duplicate rows in the metadata table
c.execute('''
    SELECT identifier, COUNT(identifier) c
    FROM metadata
    GROUP BY identifier
    HAVING c > 1
    ''')
rows = c.fetchall()
print("number of rows:",len(rows))
for row in rows:
    print(row)

In [ ]:
duplicate_ids = [r[0] for r in rows]

If we want to have a look at the duplicate entries, do the below to print out and check if the relevant fields are the same

In [ ]:
duplicates = rows

In [ ]:
sql = ('''
    SELECT id, identifier, created, cat, authors, title
    FROM metadata
    WHERE identifier IS ?
''')

for d in duplicates[:]:
    c.execute(sql, (d[0], ))
    rows = c.fetchall()
    
    same = True
    r1 = rows[0][1:]
    for row in rows[1:]:
        if r1 != row[1:]:
            same = False
    if same is False:
        print("-----",d[0])
        for row in rows:
            print(row)
        print("----- !!! mismatch !!! -----")
#     else:
#         print("*** entry identitical ***")

### Remove earlier entries for any that have duplicates

This is quick as long as there has been an index created on metadata identifiers

In [ ]:
count = 0
sql = ('''
    SELECT id, identifier
    FROM metadata
    WHERE identifier = ?
    ''')
for dup_id in duplicate_ids[:]:
    # first grab the ids of all rows in metadata table for this entry with duplicates
    c.execute(sql, (dup_id, ))
    rows = c.fetchall()
    print("number of duplicates for dup_id:",len(rows))
    for row in rows:
        print(row)
        
    print("deleting n-1 rows")
    for row in rows[:-1]:
        print(row)
        delete_sql = ("DELETE FROM metadata WHERE metadata.id = ?")
        c.execute(delete_sql, (row[0], ))
        count += 1
#         test = c.fetchall()
#         print(test)
print("finished deleting, removed {} entries".format(count))
db.commit()

## Searching JSON

JSON file provided by Kaggle - see https://www.kaggle.com/Cornell-University/arxiv
This is a single large JSON file with all metadata. Useful to double check some things as this has been cleaned up and is used more widely. Note: reading the data in will take a few minutes.

In [ ]:
data = []
filepath = os.path.expanduser("~/data/metadata/arxiv-metadata-oai-snapshot-2020-08-14.json")
with open(filepath) as json_file:
    for line in json_file:
        data.append(json.loads(line))
#     jdata = json.loads(json_file.read())

In [ ]:
def search_json(data, name):
    results = []
    for dict in data:
        if dict["id"] == name:
            results.append(dict)
    return results

In [ ]:
print(search_json(data, "0704.0342"))

In [ ]:
print(search_json(data, rows[1][0]))

## Finding missing metadata

Find where there is no matching metadata for a given image by comparing identifiers. Print and also keep this list in the `rows` array for next step.

In [ ]:
c.execute('''
    SELECT count(images.identifier), images.identifier
    FROM images
    LEFT JOIN metadata ON metadata.identifier = images.identifier
    WHERE metadata.identifier IS NULL
    GROUP BY images.identifier
    ''')
rows = c.fetchall()

print("total number of identifiers:", len(rows))
print("total number of images:", sum(row[0] for row in rows))
for row in rows:
    print(row)

Create a list with the identifiers only

In [ ]:
missing_identifiers = [x[1] for x in rows]

In [ ]:
print(len(missing_identifiers))
print(missing_identifiers)

#### Download the XML files for each of the identifiers

In [ ]:
for missing_id in missing_identifiers:
    
    URL = "http://export.arxiv.org/oai2?verb=GetRecord&identifier=oai:arXiv.org:{}&metadataPrefix=arXiv".format(missing_id)
    filepath = "extra-xml/{}.xml".format(missing_id)
    if os.path.isfile(filepath) is False:
        print("xml file doesn't exist")
        response = requests.get(URL)
        with open(filepath, 'wb') as file:
            file.write(response.content)
        time.sleep(2)
    else:
        print("xml file already exists, skipping")

This final identifier was done manually, only one old identifier
(it doesn't work because it needs a slash added to the url, this was removed in our database)

#### Iterate over all identifiers/XML files and grab key data, insert into SQLite

In [ ]:
for missing_id in missing_identifiers:
    
    filepath = "extra-xml/{}.xml".format(missing_id)

    data = ET.parse(filepath)
    root = data.getroot()
    print("opening file: " + filepath)

    OAI = "{http://www.openarchives.org/OAI/2.0/}"
    ARXIV = "{http://arxiv.org/OAI/arXiv/}"

    try:
        # get the correct nodes from XML
        request_node = root.find(OAI+'GetRecord')
        print(request_node)
        record_node = request_node.find(OAI+'record')
        print(record_node)
        metadata_node = record_node.find(OAI+'metadata')
        print(metadata_node)
        arXiv_node = metadata_node.find(ARXIV+'arXiv')
        print(arXiv_node)
        id_node = arXiv_node.find(ARXIV+'id')
        print(id_node)
        identifier = id_node.text
        print(identifier)
        # id_node = root.find(OAI+'id')

        date = arXiv_node.find(ARXIV+'created').text
        categories = arXiv_node.find(ARXIV+'categories').text
        title = arXiv_node.find(ARXIV+'title').text
        abstract = arXiv_node.find(ARXIV+'abstract').text

        license_node = arXiv_node.find(ARXIV+"license")

        if license_node is not None:
            lic = license_node.text
            # print("license_node not None")
        else:
            lic = ""

        # attempt to get authors
        authors_list = []
        authors_element = arXiv_node.find(ARXIV+"authors")

        # create a (string) variable to store all authors names
        # for now, this just writes the whole list of authors as a string
        anames = ""

        for author in authors_element:
            # print(author.find(ARXIV+"keyname").text)
            aname = ""
            kn = author.find(ARXIV+"keyname").text
            fn_node = author.find(ARXIV+"forenames")
            if fn_node is not None:
                fn = fn_node.text
            else:
                fn = ""
            aname =  kn + ", " + fn + "; "

            anames += aname

        authors_list.append(anames)

        # convert to string and remove extra characters
        # authors = "" + str(authors)
        authors = (str)(authors_list)[2:-4]

        print("-" * 20)
        print(identifier)
        print(date)
        print(categories)
        print(authors)
        print(title)
        print(abstract)
        print(lic) # don't use license as it is reserved for Python!
        print("-" * 20)

        c.execute("INSERT INTO metadata (identifier, created, cat, authors, title, abstract, licence) \
        VALUES (?, ?, ?, ?, ?, ?, ?)", \
        (identifier, date, categories, authors, title, abstract, lic))

    except KeyboardInterrupt:
        db.commit()

        # quit
        sys.exit()
    # except AttributeError as error:
        # print(error)
        # continue
    except Exception as e:
        raise e

# finally commit the changes
db.commit()

#### NB: We needed to run the code to remove slashes again after importing

## Replace a table from another database

This is used if a table has been recreated elsewhere and we would like to replace in the original database.

In [ ]:
c.execute(f'ATTACH {db_path} as arxiv')
c.execute("DROP TABLE metadata")
print("dropped table, vacuuming")
c.execute("VACUUM")
print("vacuuming done, creating new table and inserting data")
c.execute("CREATE TABLE metadata(id INTEGER PRIMARY KEY, identifier TEXT, created TEXT, cat TEXT, authors TEXT, title TEXT, abstract TEXT, licence TEXT)")
c.execute("INSERT INTO main.metadata SELECT * FROM arxiv.metadata")
print("done, committing")
db.commit()

And finally,

In [ ]:
c.close()
db.close()